In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from model_builders import GCN_siam_model
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
#import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import keras.backend as K
from NGF.preprocessing import tensorise_smiles
from custom_layers.model_creator import encode_smiles, stage_creator
from keras.layers import Dense, Dropout, Input, Lambda, concatenate,Flatten
from keras.models import Model, load_model
from data_analysis import calculate_metrics
import dill
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from distance_and_mask_fn import pairwise_distance,masked_maximum,masked_minimum

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
def triplet_loss_adapted_from_tf(y_true, y_pred):
    del y_true
    margin = 0.5
    labels = y_pred[:, :1]
    labels = tf.cast(labels, dtype='int32')
    embeddings = y_pred[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=False)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

In [3]:
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)

In [4]:
model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(96), int(104), int(120)],
        "fp_length" : [int(160), int(160), int(160)],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(512), int(512), int(256)], 
        'dropout_rate' : [0.1, 0.1],
        'lr' : 1e-4,
        'batch_size' : int(64),
        'n_epochs' : int(35),
        'margin' : 0.5
        }

In [6]:
def dataframe_to_gcn_input(input_data):
        x_atoms_cold, x_bonds_cold, x_edges_cold = tensorise_smiles(input_data,
                                                                    max_degree=model_params['max_degree'],
                                                                    max_atoms=model_params['max_atoms'])
        return [x_atoms_cold, x_bonds_cold, x_edges_cold]

In [7]:
def build_encoder(encoder_params):
        model_enc_1 = stage_creator(encoder_params, 1, conv=True)[0]
        model_enc_2 = stage_creator(encoder_params, 2, conv=True)[0]
        model_enc_3 = stage_creator(encoder_params, 3, conv=True)[0]

        model_enc_fp_1 = stage_creator(encoder_params, 1, conv=False)[1]
        model_enc_fp_2 = stage_creator(encoder_params, 2, conv=False)[1]
        model_enc_fp_3 = stage_creator(encoder_params, 3, conv=False)[1]

        atoms, bonds, edges = encode_smiles(encoder_params["max_atoms"],
                                            encoder_params["num_atom_features"],
                                            encoder_params["max_degree"],
                                            encoder_params["num_bond_features"])

        graph_conv_1 = model_enc_1([atoms, bonds, edges])
        graph_conv_2 = model_enc_2([graph_conv_1, bonds, edges])
        graph_conv_3 = model_enc_3([graph_conv_2, bonds, edges])

        fingerprint_1 = model_enc_fp_1([graph_conv_1, bonds, edges])
        fingerprint_1 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_1)

        fingerprint_2 = model_enc_fp_2([graph_conv_2, bonds, edges])
        fingerprint_2 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_2)

        fingerprint_3 = model_enc_fp_3([graph_conv_3, bonds, edges])
        fingerprint_3 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_3)

        final_fingerprint = keras.layers.add([fingerprint_1, fingerprint_2, fingerprint_3])

        return Model([atoms, bonds, edges], [final_fingerprint])

In [8]:
def build_model(model_params, encoder, verbose=False):
        atoms = Input(name='atom_inputs',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds = Input(name='bond_inputs', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges = Input(name='edge_inputs', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug = encoder([atoms, bonds, edges])

        # Fully connected
        FC1 = Dense(model_params["dense_size"][0], activation='relu',kernel_initializer='random_normal')(encode_drug)
        FC2 = Dropout(model_params["dropout_rate"][0])(FC1)
        FC2 = Dense(model_params["dense_size"][1], activation='relu',kernel_initializer='random_normal')(FC2)
        FC2 = Dropout(model_params["dropout_rate"][1])(FC2)
        FC2 = Dense(model_params["dense_size"][2], activation = None,kernel_initializer='random_normal')(FC2)
        
        
        embeddings = Lambda(lambda x: K.l2_normalize(x,axis=1),name = 'Embeddings')(FC2)
        #predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal',name = 'Predictions')(FC2)
        gcn_model = Model(inputs=[atoms, bonds, edges], outputs = embeddings)


        #if verbose:
         #   print('encoder')
          #  encoder.summary()
           # print('GCN_model')
        gcn_model.summary()
            
        return gcn_model

In [9]:
def build_mining(model_params,gcn_model):
    atoms = Input(name='atom_inputs',
                  shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
    bonds = Input(name='bond_inputs', shape=(
        model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                  dtype='float32')
    edges = Input(name='edge_inputs', shape=(model_params['max_atoms'], model_params['max_degree']),
                  dtype='int32')
    
    labels = Input(name = 'labels_inputs',shape = (1,),dtype = 'float32')
    
    encoded = gcn_model([atoms,bonds,edges])
    labels_plus_embeddings = concatenate([labels, encoded])
    
    mining_net = Model(inputs = [atoms,bonds,edges,labels],outputs = labels_plus_embeddings)
    adam = keras.optimizers.Adam(lr = model_params["lr"], 
                                 beta_1=0.9, 
                                 beta_2=0.999, 
                                 decay=0.0, 
                                 amsgrad=False)
    
    
    mining_net.compile(optimizer=adam , loss = triplet_loss_adapted_from_tf)
    mining_net.summary()
    return mining_net

In [10]:
#Old harder splits
train_0 = pd.read_csv("/Users/tomas/Documents/GitHub/kinase_binding/learning/data/p38/split_aveb/fold_0/train_0.csv",index_col=0)
val_0 = pd.read_csv("/Users/tomas/Documents/GitHub/kinase_binding/learning/data/p38/split_aveb/fold_0/val_0.csv",index_col=0)

In [11]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)


In [12]:

train = train_0#df.loc[train_val_folds[0][0]]
smiles_list = list(train.rdkit)

#Train to gcn_network
train_atoms, train_bonds, train_edges = dataframe_to_gcn_input(train['rdkit'])

Y_dummy = np.empty((train_atoms.shape[0],256+1))

In [13]:
val = val_0#df.loc[train_val_folds[0][1]]
val_atoms, val_bonds, val_edges = dataframe_to_gcn_input(val["rdkit"])
Y_dummy_val = np.empty((val_atoms.shape[0],256+1))
del train_0,val_0

In [14]:
encoder = build_encoder(model_params)
gcn_model = build_model(model_params,encoder)

LAYER 0
LAYER 1
LAYER 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs (InputLayer)        (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs (InputLayer)        (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs (InputLayer)        (None, 70, 5)        0                                            
__________________________________________________________________________________________________
model_10 (Model)                (None, 160)          241696      atom_inputs[0][0]                
                                                                 bond_inputs[0][0]   

In [15]:
mining_model = build_mining(model_params,gcn_model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs (InputLayer)        (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs (InputLayer)        (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs (InputLayer)        (None, 70, 5)        0                                            
__________________________________________________________________________________________________
labels_inputs (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
model_11 (

In [16]:
es = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)

In [18]:
mining_model.fit([train_atoms,train_bonds,train_edges,train.Binary],Y_dummy,
                 epochs = 50,
                 batch_size = 256,
                 shuffle = True,
                 validation_data=([val_atoms, val_bonds, val_edges,val.Binary],Y_dummy_val),
                 callbacks=[es,rlr2])

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2804 samples, validate on 382 samples
Epoch 1/50
2804/2804 [==============================] - 7s 2ms/step - loss: 0.4983 - val_loss: 0.4981
Epoch 2/50
2804/2804 [==============================] - 1s 268us/step - loss: 0.4981 - val_loss: 0.4978
Epoch 3/50
2804/2804 [==============================] - 1s 269us/step - loss: 0.4979 - val_loss: 0.4970
Epoch 4/50
2804/2804 [==============================] - 1s 269us/step - loss: 0.4977 - val_loss: 0.4949
Epoch 5/50
2804/2804 [==============================] - 1s 268us/step - loss: 0.4969 - val_loss: 0.4921
Epoch 6/50
2804/2804 [==============================] - 1s 270us/step - loss: 0.4953 - val_loss: 0.4875
Epoch 7/50
2804/2804 [==============================] - 1s 270us/step - loss: 0.4929 - val_loss: 0.4832
Epoch 8/50
2804/2804 [==============================] - 1s 269us/step - loss: 0.4902 - val_loss: 0.4769
Epoch 9/50
2804/2804 [==============================] - 1s 269us/step - loss: 0.4874 - val_loss: 0.4753
Epoch 10/50
2804/28

In [19]:
embeddings_val = gcn_model.predict([val_atoms, val_bonds, val_edges])
df_embeddings = pd.DataFrame(embeddings_val)
#df_embeddings.to_csv("/Users/tomas/Desktop/binding/p38_splits/embeddings_new_splits/fold_0/embeddings_val.csv")
#val.to_csv("/Users/tomas/Desktop/binding/p38_splits/embeddings_new_splits/fold_0/val_0.csv")
embeddings_train = gcn_model.predict([train_atoms, train_bonds, train_edges])
df_embeddings_train = pd.DataFrame(embeddings_train)
#df_embeddings_train.to_csv("/Users/tomas/Desktop/binding/p38_splits/embeddings_new_splits/fold_0/embeddings_train.csv")
#train.to_csv("/Users/tomas/Desktop/binding/p38_splits/embeddings_new_splits/fold_0/train_0.csv")

In [20]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [21]:
def calculate_metrics(y_true, y_pred, plots=False):
    assert isinstance(y_true, np.ndarray), 'y_true should be np.array'
    assert len(y_true.shape) == len(y_pred.shape) == 1, 'y_true or y_pred shapes are not 1 (probably not squeezed)'
    y_pred_bin = y_pred > 0.5

    cf = confusion_matrix(y_true, y_pred_bin)
    tn, fp, fn, tp = cf.ravel()

    metrics = {
        'roc_auc': roc_auc_score(y_true, y_pred),
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'map': average_precision_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred_bin),
        'recall': recall_score(y_true, y_pred_bin),
        'accuracy': accuracy_score(y_true, y_pred_bin),
    }

    if plots:
        print('predictions histogram')
        plt.figure()
        plt.hist(y_pred, bins=int(len(y_pred) / 3))
        plt.show()

        print('confusion matrix')
        plt.figure()
        group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
        group_counts = ['{0:0.0f}'.format(value) for value in
                        cf.flatten()]
        group_percentages = ['{0:.2%}'.format(value) for value in
                             cf.flatten() / np.sum(cf)]
        labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
                  zip(group_names, group_counts, group_percentages)]
        labels = np.asarray(labels).reshape(2, 2)
        sns.heatmap(cf, annot=labels, fmt='', cmap='Blues')
        plt.show()

        print('roc curve')
        random_probs = [0 for _ in range(len(y_true))]
        auc = roc_auc_score(y_true, y_pred)
        print('Logistic: ROC AUC=%.3f' % (auc))
        ns_fpr, ns_tpr, _ = roc_curve(y_true, random_probs)
        lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
        plt.plot(ns_fpr, ns_tpr, linestyle='--', label='random')
        plt.plot(lr_fpr, lr_tpr, marker='.')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.show()

    return metrics


In [22]:
hyper_params = {
        "colsample_bylevel" : 0.8252200966638428,
        "colsample_bytree" : 0.6160,
        "gamma" : 0.2070,
        "eta" : 0.5903,
        "max_delta_step" : 3,
        "max_depth" : 8,
        "min_child_weight" : 70,
        "alpha" : 21.24299,
        "lambda" : 38.191928,
        "subsample" : 0.87876195660726,
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree',
        "tree_method" : 'gpu_hist',
        "single_precision_histogram" : True
}

In [23]:
import xgboost as xgb

In [24]:
labels_binary = train.Binary
labels_val = val.Binary

In [25]:
dmatrix_train = xgb.DMatrix(data = embeddings_train,label = labels_binary)
dmatrix_val = xgb.DMatrix(data = embeddings_val,label = labels_val)

In [26]:
evalist = [(dmatrix_val,'eval'),(dmatrix_train,'train')]

In [27]:
model = xgb.train(hyper_params,dmatrix_train,300,evalist,verbose_eval=True)

[0]	eval-auc:0.57050	train-auc:0.91788
[1]	eval-auc:0.57002	train-auc:0.94278
[2]	eval-auc:0.56746	train-auc:0.95308
[3]	eval-auc:0.57031	train-auc:0.96209
[4]	eval-auc:0.58003	train-auc:0.96495
[5]	eval-auc:0.57958	train-auc:0.96853
[6]	eval-auc:0.57995	train-auc:0.97022
[7]	eval-auc:0.60167	train-auc:0.97102
[8]	eval-auc:0.62346	train-auc:0.97115
[9]	eval-auc:0.61752	train-auc:0.97167
[10]	eval-auc:0.62363	train-auc:0.97175
[11]	eval-auc:0.62471	train-auc:0.97206
[12]	eval-auc:0.62252	train-auc:0.97255
[13]	eval-auc:0.62271	train-auc:0.97263
[14]	eval-auc:0.62279	train-auc:0.97266
[15]	eval-auc:0.62365	train-auc:0.97273
[16]	eval-auc:0.62511	train-auc:0.97300
[17]	eval-auc:0.63289	train-auc:0.97318
[18]	eval-auc:0.63521	train-auc:0.97328
[19]	eval-auc:0.65243	train-auc:0.97350
[20]	eval-auc:0.65237	train-auc:0.97364
[21]	eval-auc:0.65576	train-auc:0.97363
[22]	eval-auc:0.65630	train-auc:0.97370
[23]	eval-auc:0.65212	train-auc:0.97373
[24]	eval-auc:0.65202	train-auc:0.97377
[25]	eval-

[203]	eval-auc:0.63896	train-auc:0.97493
[204]	eval-auc:0.63896	train-auc:0.97493
[205]	eval-auc:0.63896	train-auc:0.97493
[206]	eval-auc:0.63896	train-auc:0.97493
[207]	eval-auc:0.63896	train-auc:0.97493
[208]	eval-auc:0.63896	train-auc:0.97493
[209]	eval-auc:0.63896	train-auc:0.97493
[210]	eval-auc:0.63896	train-auc:0.97493
[211]	eval-auc:0.63896	train-auc:0.97493
[212]	eval-auc:0.63896	train-auc:0.97493
[213]	eval-auc:0.63896	train-auc:0.97493
[214]	eval-auc:0.63896	train-auc:0.97493
[215]	eval-auc:0.63896	train-auc:0.97493
[216]	eval-auc:0.63896	train-auc:0.97493
[217]	eval-auc:0.63896	train-auc:0.97493
[218]	eval-auc:0.63896	train-auc:0.97493
[219]	eval-auc:0.63896	train-auc:0.97493
[220]	eval-auc:0.64033	train-auc:0.97493
[221]	eval-auc:0.64033	train-auc:0.97493
[222]	eval-auc:0.64033	train-auc:0.97493
[223]	eval-auc:0.64033	train-auc:0.97493
[224]	eval-auc:0.64033	train-auc:0.97493
[225]	eval-auc:0.64033	train-auc:0.97493
[226]	eval-auc:0.64033	train-auc:0.97493
[227]	eval-auc:0

In [28]:
y_pred_val = model.predict(dmatrix_val)
y_pred_train = model.predict(dmatrix_train)

In [29]:
validation_metrics= calculate_metrics(np.array(labels_val),y_pred_val)
training_metrics = calculate_metrics(np.array(labels_binary),y_pred_train)

In [30]:
validation_metrics

{'roc_auc': 0.638676844783715,
 'tn': 233,
 'fp': 29,
 'fn': 92,
 'tp': 28,
 'map': 0.47165914144497295,
 'precision': 0.49122807017543857,
 'recall': 0.23333333333333334,
 'accuracy': 0.6832460732984293}

In [31]:
training_metrics

{'roc_auc': 0.975019210504324,
 'tn': 1212,
 'fp': 134,
 'fn': 117,
 'tp': 1341,
 'map': 0.9780558686566379,
 'precision': 0.9091525423728813,
 'recall': 0.9197530864197531,
 'accuracy': 0.9104850213980028}